[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/g-emarco/notebook_host/blob/main/pandasai_v3_showcase.ipynb)

# PandasAI v3: The Complete Showcase

This notebook provides a comprehensive overview of the capabilities of PandasAI v3, covering a wide range of functions from basic setup to advanced features like agents, custom skills, and the semantic layer.

## Section 1: Setup and Basic Chat

In [ ]:
!pip install pandasai pandasai-litellm openpyxl pandasai-openai pandasai-google-colab pandasai-lancedb pandasai-chromadb pandasai-sql pandasai-pinecone pandasai-qdrant

In [ ]:
import pandas as pd
import pandasai as pai
from pandasai_litellm.litellm import LiteLLM

# 1.1 - LiteLLM and pai.config.set
# IMPORTANT: Please add your OpenAI API key below for the notebook to run.
llm = LiteLLM(model="gpt-4o-mini", api_key="YOUR_OPENAI_API_KEY")
pai.config.set({"llm": llm})

# 1.2 - Create a sample DataFrame and use df.to_csv
data = {
    'country': ['United States', 'United Kingdom', 'France', 'Germany', 'Italy', 'Spain', 'Canada', 'Australia', 'Japan', 'China'],
    'gdp': [21.4, 2.8, 2.7, 3.8, 2.0, 1.4, 1.7, 1.4, 5.1, 14.3],
    'happiness_index': [7.0, 7.1, 6.8, 7.2, 6.5, 6.6, 7.3, 7.2, 6.9, 5.1]
}
df = pd.DataFrame(data)
df.to_csv('country_data.csv', index=False)

# 1.3 - pai.read_csv
countries_df = pai.read_csv('country_data.csv')

# 1.4 - df.chat (string output)
print(countries_df.chat('Which country has the highest GDP?'))

# 1.5 - df.chat (dataframe output)
print(countries_df.chat('What are the top 3 countries by happiness index?'))

In [ ]:
# 1.6 - df.chat (chart output) and chart_response.save
chart = countries_df.chat('Plot a bar chart of the GDP of the top 5 countries by happiness index.')
chart.save('gdp_chart.png')

## Section 2: The PandasAI Agent

In [ ]:
from pandasai import Agent

# The v3 Agent maintains conversational context automatically.
agent = Agent(countries_df, verbose=True) # Use verbose=True to see the generated code
print(agent.chat('Which are the bottom 3 countries by GDP?'))
print(agent.chat('Of these, which one has the highest happiness index?')) # No follow_up method needed

## Section 3: Custom Skills

In [ ]:
from pandasai import skill
import matplotlib.pyplot as plt

@skill()
def calculate_bonus(salary: float, performance: float) -> float:
    return salary * (performance / 100)

@skill()
def plot_salaries(names: list[str], salaries: list[float]):
    plt.bar(names, salaries)
    plt.show()

@skill()
def format_currency(amount: float) -> str:
    return f'${amount:,.2f}'

employee_data = pd.DataFrame({
    'name': ['Alice', 'Bob', 'Charlie'],
    'salary': [50000, 75000, 60000],
    'performance': [90, 85, 95]
})

employee_agent = Agent(employee_data)
employee_agent.add_skills(calculate_bonus, plot_salaries, format_currency)
print(employee_agent.chat('What is the bonus for Alice?'))
print(employee_agent.chat('Plot the salaries of the employees.'))
print(employee_agent.chat('What is the total salary formatted as currency?'))

## Section 4: LLMs and Database Connectors

In [ ]:
from pandasai.llm import OpenAI, AzureOpenAI
from pandasai.connectors import SqliteConnector, PostgreSQLConnector

# Using a functional in-memory SQLite connector
sqlite_connector = SqliteConnector(table='countries', database=':memory:')
countries_df.to_sql('countries', sqlite_connector.engine, index=False)
db_agent = Agent(sqlite_connector)
print(db_agent.chat('Which country has the highest GDP?'))

# Conceptual examples for other connectors
openai_llm = OpenAI(api_key="YOUR_OPENAI_API_KEY")
azure_llm = AzureOpenAI(deployment_name="YOUR_DEPLOYMENT_NAME", api_key="YOUR_AZURE_API_KEY", api_base="YOUR_AZURE_ENDPOINT")
pg_connector = PostgreSQLConnector(host='localhost', database='test_db', user='user', password='password')

## Section 5: Vector Stores and Agent Training

In [ ]:
from pandasai.ee.vectorstores import LanceDB, ChromaDB, Pinecone, Qdrant

# Using LanceDB
lancedb_store = LanceDB()
lance_agent = Agent(countries_df, vectorstore=lancedb_store)
lance_agent.train(queries=["What is the happiest country?"], codes=["print(df.sort_values(by='happiness_index', ascending=False).iloc[0]['country'])"])
print(lance_agent.chat("What is the happiest country?"))

# Conceptual examples for other vector stores
pinecone_store = Pinecone(api_key="YOUR_PINECONE_API_KEY")
qdrant_store = Qdrant()

## Section 6: Secure Execution

In [ ]:
!pip install pandasai-docker

In [ ]:
from pandasai_docker import DockerSandbox

sandbox = DockerSandbox()
sandbox.start()
safe_agent = Agent(countries_df, sandbox=sandbox)
result = safe_agent.chat('Plot a histogram of the happiness index.')
result.show() # Use result.show() to display the plot from the sandbox
sandbox.stop()

## Section 7: The Semantic Layer

In [ ]:
!mkdir -p my-org/datasets
semantic_df = pai.create(
    path="my-org/datasets/countries",
    df=countries_df,
    description="Data about countries, including GDP and happiness index."
)

loaded_df = pai.load("my-org/datasets/countries")
print(pai.chat('What is the GDP of the happiest country based on the loaded dataset?'))

## Section 8: Legacy v2 Compatibility

In [ ]:
from pandasai.smart_dataframe import SmartDataframe
from pandasai.smart_datalake import SmartDatalake

smart_df = SmartDataframe(countries_df, config={"llm": llm})
print(smart_df.chat('Which country has the lowest happiness index?'))

# The explain(), clarification_questions(), and rephrase_query() methods are part of the legacy SmartDataframe.
# In the v3 Agent, you can get similar information by using `verbose=True` during Agent initialization to see the generated code.
print(f'Explanation:\n{smart_df.explain()}\n')
print(f'Clarification Questions:\n{smart_df.clarification_questions("give me the top countries by gdp")}\n')
print(f'Rephrased Query:\n{smart_df.rephrase_query("give me the top countries by gdp")}\n')

lake = SmartDatalake([countries_df], config={"llm": llm})
print(lake.chat('What is the total GDP of all countries combined?'))